In [1]:
import pandas as pd
from collections import Counter
import numpy as np
from copy import deepcopy
import random
from functools import reduce
import pickle
from sklearn.model_selection import train_test_split

In [2]:
train_replace_dict = {'atis_flight#atis_airfare': 'atis_airfare', 
                                     'atis_ground_service#atis_ground_fare': 'atis_ground_fare',
                                     'atis_aircraft#atis_flight#atis_flight_no': 'atis_flight_no',
                                     'atis_airfare#atis_flight_time':'atis_flight_time',
                                     'atis_airline#atis_flight_no':'atis_flight_no',
                                     'atis_flight_no#atis_airline' : 'atis_flight_no',
                                     'atis_airfare#atis_flight': 'atis_airfare',
                                     'atis_flight#atis_airline' : 'atis_airline'}

# intent to idx mapping
intent_list = (pd.read_csv('../data/multiATIS/intent_list.csv',sep=',',header=None,names=['INTENT']).INTENT.values.tolist())
intent2idx = { intent:idx for idx,intent in enumerate(intent_list)}

# slot idx mapping
final_slots = pd.read_csv('../data/multiATIS/slots_list.csv',sep=',',header=None,names=['SLOTS']).SLOTS.values.tolist()
slots2idx  = {slots:idx for idx,slots in enumerate(final_slots)}
idx2slots = {v: k for k, v in slots2idx.items()}

In [3]:
def mapping2idx(df):
    
    #del df['id']

    # adding intent encoding
    encoded_intent = []
    for intent in list(df.INTENT):
        try:
            encoded_intent.append(intent2idx[intent])
        except:
            print(intent)
            encoded_intent.append(intent2idx['OOI'])
            
        
    df['INTENT_ID'] = encoded_intent 

    # adding mapped slot column
    slots_ID = []

    for annotations in list(df.SLOTS):
        encoded_slot = ""
        #print((annotations.split()))
        for tokens in annotations.split():
            try: 
                encoded_slot += str(slots2idx[tokens]) + " "
            except:
                print('token',tokens)
                encoded_slot += str(slots2idx['O']) + " "
        slots_ID.append(" ".join(encoded_slot.split()))

    df['SLOTS_ID'] = slots_ID
    
    return df

# Clean dataset train/val dataset variation generation

In [5]:
def process_data(data_path, train_out_path,val_out_path,test_out_path):
   
    # loading dataset
    #trainPD = pd.read_csv(data_path+'train_EN.tsv',sep='\t',header=0,names=['ID','TEXT','SLOTS','INTENT'])
    #valPD = pd.read_csv(data_path+'dev_EN.tsv',sep='\t',header=0,names=['ID','TEXT','SLOTS','INTENT'])
    testPD = pd.read_csv(data_path+'test_EN.tsv',sep='\t',header=0,names=['ID','TEXT','SLOTS','INTENT'])
    
    # lowerCasing the TEXT column
    #trainPD['TEXT'] = trainPD['TEXT'].str.lower()
    #valPD['TEXT'] = valPD['TEXT'].str.lower()
    testPD['TEXT'] = testPD['TEXT'].str.lower()
    
    # handling multi-label instances 
    #trainPD['INTENT'] = trainPD['INTENT'].replace(train_replace_dict)
    #valPD['INTENT'] = valPD['INTENT'].replace(train_replace_dict)
    testPD['INTENT'] = testPD['INTENT'].replace(train_replace_dict)
    
    #trainPD = mapping2idx(trainPD)
    #valPD = mapping2idx(valPD)
    testPD = mapping2idx(testPD)
    
    #print('Length of training set... ',len(trainPD))
    #print('Length of validation set... ',len(valPD))
    print('Length of test set... ',len(testPD))
    
    #trainPD = trainPD.sample(frac=1).reset_index(drop=True)
    #trainPD.to_csv(train_out_path,index=False,sep='\t')
    
    #valPD = valPD.sample(frac=1).reset_index(drop=True)
    #valPD.to_csv(val_out_path,index=False,sep='\t')
    
    testPD = testPD.sample(frac=1).reset_index(drop=True)
    testPD.to_csv(test_out_path,index=False,sep='\t')
    

In [6]:
process_data('../data/multiATIS/raw/train_dev_test/','../data/multiATIS/split/train/clean/train.tsv',
             '../data/multiATIS/split/valid/clean/val.tsv', '../data/multiATIS/split/test/clean/test.tsv')

atis_day_name
atis_day_name
token B-compartment
token B-stoploc.airport_code
token B-booking_class
token B-flight
Length of test set...  893
